### General

In [1]:
import os
import sys

import re
import csv

In [2]:
path = '/Users/soulofshadow/Downloads/Project'

os.chdir(path)
sys.path.append(path)

### Get dataset

#### get from hg pubmed abstract

In [3]:
from datasets import load_dataset

In [4]:
dataset_name = 'ywchoi/pubmed_abstract_0'
abstracts = load_dataset(dataset_name)

Found cached dataset parquet (/Users/soulofshadow/.cache/huggingface/datasets/ywchoi___parquet/ywchoi--pubmed_abstract_0-3c14f02f0075d77a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
abstracts = abstracts['train']['text']

In [6]:
'''
try to split a whole abstract to shorter sentence
'''

# def split_text_into_items(text, num_sentences_per_item=3):
#     # Use regular expressions to divide text into sentences
#     sentences = re.split(r'(?<=[.!?])\s', text)

#     items = []
    
#     # Organize sentences into one form for every two or three sentences.
#     for i in range(0, len(sentences), num_sentences_per_item):
#         item = ' '.join(sentences[i:i + num_sentences_per_item])
#         items.append(item)
    
#     return items


# results = []

# for abstract in abstracts:
#     result = split_text_into_items(abstract, num_sentences_per_item=3)
#     results.extend(result)

In [9]:
abstracts[0]

'An impaired heart rate recovery (HRR) has been associated with increased risk of cardiovascular events, cardiovascular, and all-cause mortality. However, the diagnostic ability of HRR for the presence and severity of coronary artery disease (CAD) has not been clearly elucidated. Our aim was to investigate the relationship between HRR and the SYNTAX (SYNergy between percutaneous coronary intervention with TAXus and cardiac surgery) score in patients with stable CAD (SCAD). A total of 406 patients with an abnormal treadmill exercise test and ≥50% coronary stenosis on coronary angiography were included. The HRR was calculated by subtracting the HR in the first minute of the recovery period from the maximum HR during exercise. The SYNTAX score ≥23 was accepted as high. Correlation of HRR with SYNTAX score and independent predictors of high SYNTAX score were determined. A high SYNTAX score was present in 172 (42%) patients. Mean HRR was lower in patients with a high SYNTAX score (9.8 ± 4.5

### MetaMap

In [6]:
from pymetamap import MetaMap
from time import sleep
# Setup UMLS Server

metamap_base_dir = path + '/public_mm'
metamap_bin_dir = '/bin/metamap18'
metamap_pos_server_dir = '/bin/skrmedpostctl'
metamap_wsd_server_dir = '/bin/wsdserverctl'

# Start servers
os.system(metamap_base_dir + metamap_pos_server_dir + ' start') # Part of speech tagger
os.system(metamap_base_dir + metamap_wsd_server_dir + ' start') # Word sense disambiguation 

# Sleep a bit to give time for these servers to start up
sleep(10)

Starting skrmedpostctl: 
started.
Starting wsdserverctl: 
started.
loading properties file /Users/soulofshadow/Downloads/Project/public_mm/WSD_Server/config/disambServer.cfg


In [7]:
mm = MetaMap.get_instance(metamap_base_dir + metamap_bin_dir)

In [8]:
keys_of_interest = ['preferred_name', 'cui', 'semtypes', 'pos_info']

def get_enti_from_mm(concepts, klist):
    entities = []
    for concept in concepts:
        if concept.__class__.__name__ == 'ConceptMMI':
            conc_dict = concept._asdict()
            conc_list = [conc_dict.get(kk) for kk in klist]
            key_value_pairs = zip(klist, conc_list)
            my_dict = {key: value for key, value in key_value_pairs}
            entities.append(my_dict)

    return entities

def remove_non_ascii(input_string):
    # Use a regular expression to match non-ASCII characters and replace them with an empty string
    return re.sub(r'[^\x00-\x7F]+', '', input_string)

In [9]:
file_path = path + '/data/build_tri/text_to_entity.csv'

def creat_csv(path):
    path  = path
    with open(path,'a+') as f:
        csv_write = csv.writer(f)
        data_row = ["text","entity"]
        csv_write.writerow(data_row)

def write_csv(data, path):
    path  = path
    with open(path,'a+') as f:
        csv_write = csv.writer(f)
        csv_write.writerow(data)

if not os.path.exists(file_path):
    creat_csv(file_path)

In [11]:
import pandas as pd
forcount = pd.read_csv(file_path)

print(len(forcount))

220240


In [ ]:
count = 0

results = abstracts[250000:]
for sentence in results:
    count += 1
    sample = [remove_non_ascii(sentence)]
    cons, errs = mm.extract_concepts(sample,
                                word_sense_disambiguation = True,
                                strict_model=True,
                                composite_phrase = 1,
                                prune = 30)

    get_ = get_enti_from_mm(cons, keys_of_interest)
    if get_:
        write_csv([sentence, get_], file_path)

    if count % 1000 == 0:
        print("Handled: ", count)